In [1]:
from pyspark.sql.functions import lit


df_url= spark.sql("SELECT * FROM Lakehouse_Projet.`4th_2024_QTRPARK CRIME REPORT`")
df_url = df_url.withColumn("YEAR", lit(2024))
df_url= df_url.withColumn("QUARTER", lit("Q4"))
display(df_url.limit(10))

StatementMeta(, 42ab24fc-aa9b-487f-b6ff-bd2f7a2f3ce4, 3, Finished, Available, Finished)

SynapseWidget(Synapse.DataFrame, 5fd0a82c-2522-41df-9668-1501c64941d3)

In [4]:
df_local= spark.sql("SELECT * FROM Lakehouse_Projet.`nyc_park_crime_stats_q3_2024 xlsx`")
df_local = df_local.withColumn("YEAR", lit(2024))
df_local= df_local.withColumn("QUARTER", lit("Q3"))
display(df_local.limit(10))

StatementMeta(, 42ab24fc-aa9b-487f-b6ff-bd2f7a2f3ce4, 6, Finished, Available, Finished)

SynapseWidget(Synapse.DataFrame, f5931179-764d-4f9f-af43-47e7128cc57f)

In [6]:
df_final=df_local.unionAll(df_url)
df_final.count()

StatementMeta(, 42ab24fc-aa9b-487f-b6ff-bd2f7a2f3ce4, 8, Finished, Available, Finished)

2310

In [28]:
# Rename columns in order to save it as a Delta Table 
df_final = df_final.withColumnRenamed('SIZE _ACRES_', 'SIZE ACRES')

def replace_spaces_in_columns(df):
    for col in df.columns:
        if " " in col:
            df = df.withColumnRenamed(col, col.replace(" ", "_"))
    return df

df_final = replace_spaces_in_columns(df_final)
df_final.printSchema()
df_final.show(5)

StatementMeta(, 42ab24fc-aa9b-487f-b6ff-bd2f7a2f3ce4, 30, Finished, Available, Finished)

root
 |-- PARK: string (nullable = true)
 |-- BOROUGH: string (nullable = true)
 |-- SIZE_ACRES: double (nullable = true)
 |-- CATEGORY: string (nullable = true)
 |-- MURDER: long (nullable = true)
 |-- RAPE: long (nullable = true)
 |-- ROBBERY: long (nullable = true)
 |-- FELONY_ASSAULT: long (nullable = true)
 |-- BURGLARY: long (nullable = true)
 |-- GRAND_LARCENY: long (nullable = true)
 |-- GRAND_LARCENY_OF_MOTOR_VEHICLE: long (nullable = true)
 |-- TOTAL: long (nullable = true)
 |-- YEAR: integer (nullable = false)
 |-- QUARTER: string (nullable = false)

+--------------------+-------------+----------+--------------------+------+----+-------+--------------+--------+-------------+------------------------------+-----+----+-------+
|                PARK|      BOROUGH|SIZE_ACRES|            CATEGORY|MURDER|RAPE|ROBBERY|FELONY_ASSAULT|BURGLARY|GRAND_LARCENY|GRAND_LARCENY_OF_MOTOR_VEHICLE|TOTAL|YEAR|QUARTER|
+--------------------+-------------+----------+--------------------+------+---

In [29]:
output_path = "/lakehouse/default/Tables/Silver/nyc_park_crime_concatenated.csv"
output_path_simple = "nyc_park_crime_delta"

df_final.write.format("delta").mode("overwrite").saveAsTable(output_path_simple)



StatementMeta(, 42ab24fc-aa9b-487f-b6ff-bd2f7a2f3ce4, 31, Finished, Available, Finished)

In [1]:
df_table= spark.sql("SELECT * FROM Lakehouse_Projet.`nyc_park_crime_delta`")

display(df_table.limit(10))

StatementMeta(, 82e809d7-3c31-4e3f-8e58-00a8bdb401c1, 3, Finished, Available, Finished)

SynapseWidget(Synapse.DataFrame, 0df5d893-9990-4d77-87e6-f8541d113459)